## Evaluate no_spinup column-based NNs

Usually:
- Load data (Has spin-up removed, is downsampled)
- Split into train/valid
- Normalize validation set
- Predict on validation set

Here:
- Load data
- Remove spin-up
- Normalize all data [Cannot downsample yet as it is cell-based]
- Predict on all data
- Downsample [Have to downsample before doing the train/validation split!]
- Split into train/validation set
- Compute MSE/R2 on validation set

In [21]:
# Ran with 800GB (750GB should also be fine)

import sys
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import gc

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation

# For Leaky_ReLU:
from tensorflow import nn 

t0 = time.time()
path = '/home/b/b309170'

# Add path with my_classes to sys.path
sys.path.insert(0, path + '/workspace_icon-ml/cloud_cover_parameterization/')

# Reloading custom file to incorporate changes dynamically
import importlib
import my_classes
importlib.reload(my_classes)

from my_classes import write_infofile
from my_classes import read_mean_and_std
from my_classes import TimeOut

# Cross-validation fold (in 0,1,2)
# fold = int(sys.argv[1]) #!
fold = 1

# Set seed for reproducibility
seed = 10
tf.random.set_seed(seed)

# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[3], 'GPU')

print(tf.__version__)

2.4.1


In [22]:
# Cloud Cover or Cloud Area?
# output_var = sys.argv[3] # Set output_var to one of {'cl_volume', 'cl_area'} #!
output_var = 'cl_area'

path_base = os.path.join(path, 'workspace_icon-ml/cloud_cover_parameterization/grid_column_based_DYAMOND')
path_data = os.path.join(path, 'my_work/icon-ml_data/cloud_cover_parameterization/grid_column_based_DYAMOND')
    
path_model = os.path.join(path_base, 'saved_models')
path_figures = os.path.join(path_base, 'figures')

In [23]:
# Won't run on a CPU node
try:
    # Prevents crashes of the code
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
    # Allow the growth of memory Tensorflow allocates (limits memory usage overall)
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except:
    pass

### Load the data

In [25]:
input_data = np.transpose(np.load(path_data + '/cloud_cover_input_dyamond.npy'))

# if output_var == 'cl_volume':
#     output_data = np.transpose(np.load(path_data + '/cloud_cover_output_dyamond.npy'))
# elif output_var == 'cl_area':
#     output_data = np.transpose(np.load(path_data + '/cloud_area_output_dyamond.npy'))

## Remove the spin-up & reshape back (467*79342, 163)
# Actually, we have to remove the spinup here, if we want to have a model comparable to the other ones from symbolic regression!
t_steps = 619
h_fields = 79342
no_vars = 163

## For the input data
B = np.zeros((t_steps, no_vars, h_fields))
# Invert reshaping
for i in range(no_vars):
    B[:, i] = np.reshape(input_data[:, i], (t_steps, h_fields))
# Discard spinup
input_data = np.concatenate((B[80:329], B[(329+72):]), axis=0)

# Reshape back
B = [np.reshape(input_data[:, i], -1) for i in range(no_vars)]
input_data = np.array(B).T

# no_vars = 27

# ## For the output data
# B = np.zeros((t_steps, no_vars, h_fields))
# # Invert reshaping
# for i in range(no_vars):
#     B[:, i] = np.reshape(output_data[:, i], (t_steps, h_fields))
# # Discard spinup
# output_data = np.concatenate((B[80:329], B[(329+72):]), axis=0)

# # Reshape back
# B = [np.reshape(output_data[:, i], -1) for i in range(no_vars)]
# output_data = np.array(B).T

In [26]:
(samples_total, no_of_features) = input_data.shape
(samples_total, no_of_features)

(37052714, 163)

Remove columns that are constant in at least one of the training folds

In [27]:
remove_fields = [27, 28, 29, 30, 31, 32, 135, 136, 137]
assert no_of_features == 163
input_data = np.delete(input_data, remove_fields, axis=1)
no_of_features = no_of_features - len(remove_fields)

### Define the model

Activation function for the last layer

In [28]:
custom_objects = {}
custom_objects['leaky_relu'] = nn.leaky_relu

In [29]:
model_name = 'cross_validation_column_based_%s_fold_%d_no_spinup.h5'%(output_var, (fold+1))
model = load_model(os.path.join(path_model, model_name), custom_objects)

#### Useful functions to plot results

In [40]:
def predict(model, input_data, batch_size=2**20):
    '''
        Model predictions
    '''        
    for i in range(input_data.shape[0]//batch_size + 1): 
        if i == 0:
            predictions = model.predict_on_batch(input_data[i*batch_size:(i+1)*batch_size])
        else:
            predictions = np.concatenate((predictions, model.predict_on_batch(input_data[i*batch_size:(i+1)*batch_size])), axis=0)
        K.clear_session()
        gc.collect()
        
    return predictions

#### Evaluate the models on the data

Add training and validation losses to the text files. <br>
Print results per vertical layer (respective validation set)

In [47]:
train_losses = [] ; valid_losses = [] ; valid_means = [] ; valid_model_predictions = []
t_steps = 467; h_fields = 79342

filename = 'cross_validation_column_based_%s_fold_%d_no_spinup'%(output_var, (fold+1))
    
#Standardize according to the fold
mean, std = read_mean_and_std(os.path.join(path_model, filename+'.txt'))

#Load the data for the respective fold
input_scaled = (input_data - mean)/std
predictions = predict(model, input_scaled)

## Reshape and downsample
pred_reshaped = np.zeros((t_steps, 27, h_fields))

for i in range(27):
    pred_reshaped[:, i] = np.reshape(predictions[:, i], (t_steps, h_fields))
    
pred_reshaped = np.reshape(pred_reshaped, -1)

#Load indices that we should keep
inds = np.load('~/my_work/icon-ml_data/cloud_cover_parameterization/neighborhood_based_SR_DYAMOND/indices_to_keep_after_downsampling.npy')
if output_var == 'cl_area':
    output_data = np.load('~/my_work/icon-ml_data/cloud_cover_parameterization/neighborhood_based_SR_DYAMOND/cloud_area_output_dyamond.npy')

# The maximum index to keep should be close to the total number of indices
assert (np.max(inds) - len(pred_reshaped)) < 100 
    
pred_reshaped = pred_reshaped[inds]

assert len(pred_reshaped) == len(output_data)
samples_total = len(pred_reshaped)

# Split into training/validation sets
training_folds = []
validation_folds = []
two_week_incr = samples_total//6

for i in range(3):
    # Note that this is a temporal split since time was the first dimension in the original tensor
    first_incr = np.arange(samples_total//6*i, samples_total//6*(i+1))
    second_incr = np.arange(samples_total//6*(i+3), samples_total//6*(i+4))

    validation_folds.append(np.append(first_incr, second_incr))
    training_folds.append(np.arange(samples_total))
    training_folds[i] = np.delete(training_folds[i], validation_folds[i])
    
# Validation data
output_train = output_data[training_folds[1]]
output_valid = output_data[validation_folds[1]]
preds_train = pred_reshaped[training_folds[1]]
preds_valid = pred_reshaped[validation_folds[1]]

# Losses
mse_train_unbounded = np.mean((output_train - preds_train)**2, dtype=np.float64)
mse_valid_unbounded = np.mean((output_valid - preds_valid)**2, dtype=np.float64)
mse_train_bounded = np.mean((output_train - np.minimum(np.maximum(preds_train, 0), 100))**2, dtype=np.float64)
mse_valid_bounded = np.mean((output_valid - np.minimum(np.maximum(preds_valid, 0), 100))**2, dtype=np.float64)

# Save output
with open(os.path.join(path_model, filename+'.txt'), 'a') as file:
    file.write('Unbounded training loss: %.4f\n'%(mse_train_unbounded))
    file.write('Unbounded validation loss: %.4f\n'%(mse_valid_unbounded))
    file.write('Bounded training loss: %.4f\n'%(mse_train_bounded))
    file.write('Bounded validation loss: %.4f\n'%(mse_valid_bounded))